# Bay Area Bike Share Analysis

### Reading data

Before making the hypothesis, it is always a good practice to look at the snapshot of data in order to understand the various attributes

In [1]:
from delorean import parse
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('data/trip_data.csv', parse_dates=True)

In [3]:
df.shape

(669959, 11)

In [4]:
df.head(20)

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscription Type,Zip Code
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103
5,4927,103,8/29/2013 18:54,Golden Gate at Polk,59,8/29/2013 18:56,Golden Gate at Polk,59,527,Subscriber,94109
6,4500,109,8/29/2013 13:25,Santa Clara at Almaden,4,8/29/2013 13:27,Adobe on Almaden,5,679,Subscriber,95112
7,4563,111,8/29/2013 14:02,San Salvador at 1st,8,8/29/2013 14:04,San Salvador at 1st,8,687,Subscriber,95112
8,4760,113,8/29/2013 17:01,South Van Ness at Market,66,8/29/2013 17:03,South Van Ness at Market,66,553,Subscriber,94103
9,4258,114,8/29/2013 11:33,San Jose City Hall,10,8/29/2013 11:35,MLK Library,11,107,Subscriber,95060


In [5]:
df.dtypes

Trip ID               int64
Duration              int64
Start Date           object
Start Station        object
Start Terminal        int64
End Date             object
End Station          object
End Terminal          int64
Bike #                int64
Subscription Type    object
Zip Code             object
dtype: object

In [6]:
df["Subscription Type"].value_counts()

Subscriber    566746
Customer      103213
Name: Subscription Type, dtype: int64

### Hypothesis

Based on the data, what are the questions we can answer?

1. Given a time slot and terminal code, predict if the terminal has high traffic
2. Given a time slot and terminal code, perdict number of transactions

## Hypothesis 1

### Given a time slot and terminal code, predict if the terminal has high traffic

### Feature engineering

* Find & replace null values
* Hour window
* Target generation
* Removal of non categorical fields

In [7]:
pd.isnull(df["Start Date"]).all()

False

In [8]:
def get_hour(value):
    
    parsed_date = parse(value).datetime.hour
    return parsed_date
    

In [9]:
df["start_hour"] = df["Start Date"].apply(get_hour)

In [10]:
df.start_hour.value_counts()

8     85864
17    82705
9     62897
16    59099
18    57652
7     43939
12    34384
15    33223
13    31740
10    30106
19    29188
11    29141
14    27156
20    16527
6     14312
21    11277
22     7434
23     4450
5      3449
0      2171
1      1189
4      1022
2       692
3       342
Name: start_hour, dtype: int64

In [11]:
a = parse('7/25/2016 14:13')

In [12]:
a.datetime.isoweekday()

1

In [13]:
def get_day(value):
    
    parsed_date = parse(value).datetime.isoweekday()
    return parsed_date

In [14]:
df["start_day"] = df["Start Date"].apply(get_day)

In [15]:
df.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscription Type,Zip Code,start_hour,start_day
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127,14,4
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138,14,4
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214,10,4
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060,11,4
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103,12,4


In [16]:
df["end_hour"] = df["End Date"].apply(get_hour)

In [17]:
df["end_day"] = df["End Date"].apply(get_day)

In [18]:
df.head()

,Trip ID,Duration,Start Date,Start Station,Start Terminal,End Date,End Station,End Terminal,Bike #,Subscription Type,Zip Code,start_hour,start_day,end_hour,end_day
0,4576,63,8/29/2013 14:13,South Van Ness at Market,66,8/29/2013 14:14,South Van Ness at Market,66,520,Subscriber,94127,14,4,14,4
1,4607,70,8/29/2013 14:42,San Jose City Hall,10,8/29/2013 14:43,San Jose City Hall,10,661,Subscriber,95138,14,4,14,4
2,4130,71,8/29/2013 10:16,Mountain View City Hall,27,8/29/2013 10:17,Mountain View City Hall,27,48,Subscriber,97214,10,4,10,4
3,4251,77,8/29/2013 11:29,San Jose City Hall,10,8/29/2013 11:30,San Jose City Hall,10,26,Subscriber,95060,11,4,11,4
4,4299,83,8/29/2013 12:02,South Van Ness at Market,66,8/29/2013 12:04,Market at 10th,67,319,Subscriber,94103,12,4,12,4


In [38]:
start_df = df.groupby(by=["start_hour", "start_day", "Start Terminal"]).count().copy()
start_df = start_df.reset_index()

In [39]:
start_df = start_df.ix[:, ["start_hour", "start_day", "Start Terminal", "Trip ID"]]
start_df.columns = ["hour", "day", "terminal_code", "trip_id"]
start_df.head()

,hour,day,terminal_code,trip_id
0,0,1,2,3
1,0,1,3,3
2,0,1,4,2
3,0,1,7,2
4,0,1,8,2


In [40]:
end_df = df.groupby(by=["end_hour", "end_day", "End Terminal"]).count().copy()
end_df = end_df.reset_index()
end_df = end_df.ix[:, ["end_hour", "end_day", "End Terminal", "Trip ID"]]
end_df.columns = ["hour", "day", "terminal_code", "trip_id"]
end_df.head()

,hour,day,terminal_code,trip_id
0,0,1,2,2
1,0,1,3,8
2,0,1,4,7
3,0,1,5,1
4,0,1,6,1


In [58]:
merged_df = start_df.merge(end_df, how="inner", on=["hour", "day", "terminal_code"])

In [59]:
merged_df.head()

,hour,day,terminal_code,trip_id_x,trip_id_y
0,0,1,2,3,2
1,0,1,3,3,8
2,0,1,4,2,7
3,0,1,7,2,12
4,0,1,8,2,2


In [60]:
merged_df["trip_count"] = merged_df["trip_id_x"] + merged_df["trip_id_y"]

In [61]:
merged_df = merged_df.ix[:, ["hour", "day", "terminal_code", "trip_count"]]

In [62]:
merged_df.head()

,hour,day,terminal_code,trip_count
0,0,1,2,5
1,0,1,3,11
2,0,1,4,9
3,0,1,7,14
4,0,1,8,4


In [68]:
merged_df.trip_count.mean()

144.0308788598575

In [69]:
merged_df["target"] = 0
merged_df.ix[(merged_df.trip_count > merged_df.trip_count.mean()), "target"] = 1

In [70]:
merged_df.target.value_counts()

0    6451
1    2811
Name: target, dtype: int64

In [71]:
merged_df.head()

,hour,day,terminal_code,trip_count,target
0,0,1,2,5,0
1,0,1,3,11,0
2,0,1,4,9,0
3,0,1,7,14,0
4,0,1,8,4,0
